In [1]:
from model.ResNet34 import resnet34_1d
import torch
from util.util import load_data_percentage, CustomDataset
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torch

model = resnet34_1d()
model

ResNet1D(
  (conv1): Conv1d(2, 64, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ELU): ELU(alpha=1.0, inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (ELU): ELU(alpha=1.0, inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (ELU): ELU(alpha=1.0, inplace=True)
      (conv2): Conv1d

In [2]:
# Set device
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data
X, Y, Z = load_data_percentage('./data/X.npy', './data/Y.npy', './data/Z.npy', percentage=100)
dataset = CustomDataset(X, Y, Z)
# X is of shape (18480, 32768, 2)
# Y is of shape (18480, 32768, 1)
# Z is of shape (18480, 1, 2)

# Create DataLoader
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss, and optimizer
model = resnet34_1d().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 250
with open("log/loss.txt", "a") as log_file:  # Open log file in append mode
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch_idx, (x, _, z) in enumerate(train_loader):
            x, z = x.to(device), z.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, z)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            if (batch_idx + 1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
        
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')
        
        # Log the average loss of each epoch to the file
        log_file.write(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}\n')

print("Training finished!")

# Simple evaluation
model.eval()
with torch.no_grad():
    x, _, z = next(iter(train_loader))
    x, z = x.to(device), z.to(device)
    outputs = model(x)
    eval_loss = criterion(outputs, z)
    print(f"Evaluation Loss: {eval_loss.item():.4f}")

# Save model weights
model_name = f"./weights/resnet34_{epoch}.pth"
torch.save(model.state_dict(), model_name)

Using device: cuda:3
Epoch [1/2], Average Loss: 50317.5488
Epoch [2/2], Average Loss: 49242.0964
Training finished!
Evaluation Loss: 48176.3789
